# LLM 과 OpenSearch 를 이용해서 RAG 아키텍처를 구축합니다.


여기서는 OpenSearch 가 설치된 것을 가정합니다.

## 0. 필요한 변수 불러오기

In [ ]:
# OpenSearch와 통신하기 위해서 앞서 저장했던 변수를 불러옵니다.
%store -r opensearch_user_id opensearch_user_password opensearch_domain_name opensearch_domain_endpoint

print(f"OpenSearch User Id: {opensearch_user_id}")
print(f"OpenSearch User Password: {opensearch_user_password}")
print(f"OpenSearch Domain Name: {opensearch_domain_name}")
print(f"OpenSearch Domain Endpoint: {opensearch_domain_endpoint}")
print()
    
    
# SageMaker Endpoint 와 통신하기 위해서 저장했던 변수를 불러옵니다.
%store -r endpoint_name
print(f"SageMaker EndpointName: {endpoint_name}")
print()

In [ ]:
# 저장이 되어 있지 않다면 직접 관련 정보를 입력합니다.

# opensearch_user_id = 'raguser'
# opensearch_user_password = 'Passw0rd1!'

# opensearch_domain_name = 'rag-hol-6efd6d14'
# opensearch_domain_endpoint = 'https://search-rag-hol-6efd6d14-5wbkv7qjrlci47h5ka63cw5fxy.us-west-2.es.amazonaws.com'

# endpoint_name='llama-3-2-1b-instruct-2024-11-18-14-18-13-972'

## 1. Bedrock 호출 테스트

In [ ]:
import boto3
import json

# Bedrock 클라이언트 생성
bedrock_runtime = boto3.client('bedrock-runtime')

# converse API 호출
response = bedrock_runtime.converse(
    modelId="anthropic.claude-3-haiku-20240307-v1:0",
    messages=[{"role": "user", "content": [{"text": "안녕하세요, 당신은 누구인가요?"}]}],
    system=[{"text": "당신은 친절한 AI 친구입니다. 상냥하게 답변해주세요."}],
    inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9}
)

# 응답 처리
ai_message = response["output"]["message"]
output_text = ai_message["content"][0]["text"]
print(f"{output_text}")

## 2. SageMaker Endpoint (llama 3.2 1b - instruct) 호출 테스트

In [ ]:
import json
import boto3
import datetime

region = boto3.Session().region_name
sagemaker_runtime = boto3.client(
    "sagemaker-runtime",
    region_name=region
)

In [ ]:
client = boto3.client("sagemaker-runtime", region_name='us-west-2')

system ="You are a helpful assistant" 
text = "Hello! Who are you?"
prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{system}|eot_id|><|start_header_id|>user<|end_header_id|>

{text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

payload = {
    "inputs": prompt,
    "parameters": {
        "max_new_tokens": 128,
        "top_p": 0.9,
        "temperature": 0.9,
        # "stream": True,
        # "return_full_text": False
    }
}

time_to_first_token = -1
start_time = datetime.datetime.now()

response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=json.dumps(payload),
)

# 응답 처리
end_time = datetime.datetime.now()
response_body = json.loads(response['Body'].read().decode())

# 응답 시간 계산
total_time = (end_time - start_time).total_seconds()

# 결과 출력
print(f"# System\n- {system}\n")
print(f"# Text\n- {text}\n")
print(f"# Prompt\n- {prompt}")
print(f"# Response\n- {response_body['generated_text']}\n")
print(f"# Total time\n- {total_time:.2f} seconds\n")

## 3. OpenSearch 호출 테스트 - 문서 검색

query = "아마존의 비전은 무엇인가요"
k = 3
results=vector_db.similarity_search_with_score(
    query,
    k=k
)
page_contents = [res[0].page_content for res in results]
print(page_contents[0])

In [ ]:
# from opensearchpy import OpenSearch, RequestsHttpConnection

# # Delete index
# os_client = OpenSearch(
#     region=region,
#     hosts=[{'host': opensearch_domain_endpoint.replace("https://", ""), 'port': 443}],
#     http_auth=http_auth,
#     use_ssl=True,
#     verify_certs=True,
#     connection_class=RequestsHttpConnection
# )
# os_client.indices.delete(index=index_name)

## 4. RAG 구성으로 호출 - Bedrock 사용해서 응답 생성

### 4-1. Knowledge Base 에서 질문과 유사도가 높은 내용 검색

In [ ]:
query = "아마존의 비전은 무엇인가요"
k = 3
results=vector_db.similarity_search_with_score(
    query,
    k=k
)
page_contents = [res[0].page_content for res in results]
context = page_contents[0]
print(context)

### 4-2. 유사도가 높은 내용(Context)을 포함한 프롬프트 작성

In [ ]:
prompt = f"""
<context> 태그 안의 내용을 참고해서 사용자 질문에 답변해줘.
서두는 건너뛰고 질문에 대한 답변만 해줘

<context>
{context}
</context>

User: {query}
Assistant:
"""

prompt

### 4-2. 응답 생성

In [ ]:
import boto3
import json

# Bedrock 클라이언트 생성
bedrock_runtime = boto3.client('bedrock-runtime')

# converse API 호출
response = bedrock_runtime.converse(
    modelId="anthropic.claude-3-haiku-20240307-v1:0",
    messages=[{"role": "user", "content": [{"text": prompt}]}],
    system=[{"text": "당신은 친절한 AI 친구입니다. 상냥하게 답변해주세요."}],
    inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9}
)

# 응답 처리
ai_message = response["output"]["message"]
output_text = ai_message["content"][0]["text"]
print(f"{output_text}")

## 5. RAG 구성으로 호출 - Llama 3.2 사용해서 응답 생성

### 5-1. Knowledge Base 에서 질문과 유사도가 높은 내용 검색

In [ ]:
query = "What is Amazon's vision?"
k = 3
results=vector_db.similarity_search_with_score(
    query,
    k=k
)
page_contents = [res[0].page_content for res in results]
print(page_contents[0])

### 5-2. 유사도가 높은 내용(Context)을 포함한 프롬프트 작성

In [ ]:
prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Given the following information about user question.
{system}|eot_id|><|start_header_id|>user<|end_header_id|>
{text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

In [ ]:
import json
import boto3
import datetime

region = boto3.Session().region_name
sagemaker_runtime = boto3.client(
    "sagemaker-runtime",
    region_name=region
)

In [ ]:
client = boto3.client("sagemaker-runtime", region_name='us-west-2')

# Payload 직성
payload = {
    "inputs": prompt,
    "parameters": {
        "max_new_tokens": 512,
        "top_p": 0.9,
        "temperature": 0.9,
    }
}

# SageMaer Endpoint 호출
response = client.invoke_endpoint(
    # EndpointName=endpoint_name,
    EndpointName="llama-3-2-1b-instruct-2024-11-18-14-18-13-972",
    ContentType="application/json",
    Body=json.dumps(payload),
)

# 응답 처리
end_time = datetime.datetime.now()
response_body = json.loads(response['Body'].read().decode())


# 결과 출력
print(f"# Response\n- {response_body['generated_text']}\n")